# 
Sugarscape Constant Growback Model
================================

Replication of the model found in Netlogo:
Li, J. and Wilensky, U. (2009). NetLogo Sugarscape 2 Constant Growback model.
http://ccl.northwestern.edu/netlogo/models/Sugarscape2ConstantGrowback.
Center for Connected Learning and Computer-Based Modeling,
Northwestern University, Evanston, IL.


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14
import os
import sys
import glob
import time
import warnings
import datetime 
print("Last updated on ", time.asctime())
import numpy as np
import pandas as pd
import matplotlib

Last updated on  Sat Apr 18 13:26:07 2020


In [2]:
fontsize=20
font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': fontsize,
        }

plt.rcParams["font.size"     ] = fontsize


In [9]:
from collections import defaultdict

In [117]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.visualization.TextVisualization import TextVisualization, TextGrid

In [42]:
from random import Random

# Agents

In [121]:
def get_distance(pos_1, pos_2):
    """ Get the distance between two point

    Args:
        pos_1, pos_2: Coordinate tuples for both points.

    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return np.sqrt(dx ** 2 + dy ** 2)


class SsAgent(Agent):
    def __init__(self, pos, model, moore=False, sugar=0, metabolism=0, vision=0):
        super().__init__(pos, model)
        self.pos = pos
        self.moore = moore
        self.sugar = sugar
        self.metabolism = metabolism
        self.vision = vision

    def get_sugar(self, pos):
        this_cell = self.model.grid.get_cell_list_contents([pos])
        for agent in this_cell:
            if type(agent) is Sugar:
                return agent

    def is_occupied(self, pos):
        this_cell = self.model.grid.get_cell_list_contents([pos])
        return len(this_cell) > 1

    def move(self):
        # Get neighborhood within vision
        neighbors = [i for i in self.model.grid.get_neighborhood(self.pos, self.moore,
                False, radius=self.vision) if not self.is_occupied(i)]
        neighbors.append(self.pos)
        # Look for location with the most sugar
        max_sugar = max([self.get_sugar(pos).amount for pos in neighbors])
        candidates = [pos for pos in neighbors if self.get_sugar(pos).amount ==
                max_sugar]
        # Narrow down to the nearest ones
        min_dist = min([get_distance(self.pos, pos) for pos in candidates])
        final_candidates = [pos for pos in candidates if get_distance(self.pos,
            pos) == min_dist]
        self.random.shuffle(final_candidates)
        self.model.grid.move_agent(self, final_candidates[0])

    def eat(self):
        sugar_patch = self.get_sugar(self.pos)
        self.sugar = self.sugar - self.metabolism + sugar_patch.amount
        sugar_patch.amount = 0

    def step(self):
        self.move()
        self.eat()
        if self.sugar <= 0:
            self.model.grid._remove_agent(self.pos, self)
            self.model.schedule.remove(self)


class Sugar(Agent):
    def __init__(self, pos, model, max_sugar):
        super().__init__(pos, model)
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):
        self.amount = min([self.max_sugar, self.amount + 1])


# Model

In [125]:
class SugarscapeCg(Model):
    '''
    Sugarscape 2 Constant Growback
    '''

    verbose = True  # Print-monitoring

    def __init__(self, height=50, width=50,
                 initial_population=100):
        '''
        Create a new Constant Growback model with the given parameters.

        Args:
            initial_population: Number of population to start with
        '''

        # Set parameters
        self.height = height
        self.width = width
        self.initial_population = initial_population

        self.schedule = RandomActivationByBreed(self)
        self.grid = MultiGrid(self.height, self.width, torus=False)
        self.datacollector = DataCollector({"SsAgent": lambda m: m.schedule.get_breed_count(SsAgent), })

        # Create sugar

        sugar_distribution = np.genfromtxt("sugar-map-10x12.txt")
        print(f'sugar map ->{sugar_distribution}')
        for _, x, y in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            sugar = Sugar((x, y), self, max_sugar)
            print(f'Place amount {max_sugar} of sugar in ({x,y})')
            self.grid.place_agent(sugar, (x, y))
            self.schedule.add(sugar)

        # Create agent:
        for i in range(self.initial_population):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            sugar = self.random.randrange(6, 25)
            metabolism = self.random.randrange(2, 4)
            vision = self.random.randrange(1, 6)
            print(f'Place ant in ({x,y})')
            ssa = SsAgent((x, y), self, False, sugar, metabolism, vision)
            self.grid.place_agent(ssa, (x, y))
            self.schedule.add(ssa)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print([self.schedule.time,
                   self.schedule.get_breed_count(SsAgent)])

    def run_model(self, step_count=200):

        if self.verbose:
            print('Initial number Sugarscape Agent: ',
                  self.schedule.get_breed_count(SsAgent))

        for i in range(step_count):
            self.step()

        if self.verbose:
            print('')
            print('Final number Sugarscape Agent: ',
                  self.schedule.get_breed_count(SsAgent))


In [127]:
sg = SugarscapeCg(height=12, width=10,
                 initial_population=10)

sugar map ->[[1. 1. 1. 2. 2. 2. 2. 2. 2. 2.]
 [1. 1. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
 [2. 2. 2. 2. 2. 2. 2. 3. 3. 3.]
 [2. 2. 2. 2. 2. 2. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 2. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 3. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 3. 3. 3. 3. 3. 4.]
 [2. 2. 2. 2. 3. 3. 3. 3. 3. 4.]]
Place amount 1.0 of sugar in ((0, 0))
Place amount 1.0 of sugar in ((0, 1))
Place amount 1.0 of sugar in ((0, 2))
Place amount 2.0 of sugar in ((0, 3))
Place amount 2.0 of sugar in ((0, 4))
Place amount 2.0 of sugar in ((0, 5))
Place amount 2.0 of sugar in ((0, 6))
Place amount 2.0 of sugar in ((0, 7))
Place amount 2.0 of sugar in ((0, 8))
Place amount 2.0 of sugar in ((0, 9))
Place amount 1.0 of sugar in ((1, 0))
Place amount 1.0 of sugar in ((1, 1))
Place amount 2.0 of sugar in ((1, 2))
Place amount 2.0 of sugar in ((1, 3))
Place amount 2.0 of sugar in ((1, 4))
Place amount 2.0 of s

IndexError: list index out of range

In [130]:
class SugarscapeCgFix(Model):
    '''
    Sugarscape 2 Constant Growback
    '''

    verbose = True  # Print-monitoring

    def __init__(self, height=50, width=50,
                 initial_population=100):
        '''
        Create a new Constant Growback model with the given parameters.

        Args:
            initial_population: Number of population to start with
        '''

        # Set parameters
        self.height = height
        self.width = width
        self.initial_population = initial_population

        self.schedule = RandomActivationByBreed(self)
        self.grid = MultiGrid(self.height, self.width, torus=False)
        self.datacollector = DataCollector({"SsAgent": lambda m: m.schedule.get_breed_count(SsAgent), })

        # Create sugar

        sugar_distribution = np.genfromtxt("sugar-map-10x12.txt")
        print(f'sugar map ->{sugar_distribution}')
        for _, x, y in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            sugar = Sugar((x, y), self, max_sugar)
            print(f'Place amount {max_sugar} of sugar in ({x,y})')
            self.grid.place_agent(sugar, (x, y))
            self.schedule.add(sugar)

        # Create agent:
        for i in range(self.initial_population):
            x = self.random.randrange(self.height)
            y = self.random.randrange(self.width)
            sugar = self.random.randrange(6, 25)
            metabolism = self.random.randrange(2, 4)
            vision = self.random.randrange(1, 6)
            print(f'Place ant in ({x,y})')
            ssa = SsAgent((x, y), self, False, sugar, metabolism, vision)
            self.grid.place_agent(ssa, (x, y))
            self.schedule.add(ssa)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.verbose:
            print([self.schedule.time,
                   self.schedule.get_breed_count(SsAgent)])

    def run_model(self, step_count=200):

        if self.verbose:
            print('Initial number Sugarscape Agent: ',
                  self.schedule.get_breed_count(SsAgent))

        for i in range(step_count):
            self.step()

        if self.verbose:
            print('')
            print('Final number Sugarscape Agent: ',
                  self.schedule.get_breed_count(SsAgent))


In [131]:
sg = SugarscapeCgFix(height=12, width=10,
                 initial_population=10)

sugar map ->[[1. 1. 1. 2. 2. 2. 2. 2. 2. 2.]
 [1. 1. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2. 2. 2. 2. 3. 3.]
 [2. 2. 2. 2. 2. 2. 2. 3. 3. 3.]
 [2. 2. 2. 2. 2. 2. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 2. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 2. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 3. 3. 3. 3. 3. 3.]
 [2. 2. 2. 2. 3. 3. 3. 3. 3. 4.]
 [2. 2. 2. 2. 3. 3. 3. 3. 3. 4.]]
Place amount 1.0 of sugar in ((0, 0))
Place amount 1.0 of sugar in ((0, 1))
Place amount 1.0 of sugar in ((0, 2))
Place amount 2.0 of sugar in ((0, 3))
Place amount 2.0 of sugar in ((0, 4))
Place amount 2.0 of sugar in ((0, 5))
Place amount 2.0 of sugar in ((0, 6))
Place amount 2.0 of sugar in ((0, 7))
Place amount 2.0 of sugar in ((0, 8))
Place amount 2.0 of sugar in ((0, 9))
Place amount 1.0 of sugar in ((1, 0))
Place amount 1.0 of sugar in ((1, 1))
Place amount 2.0 of sugar in ((1, 2))
Place amount 2.0 of sugar in ((1, 3))
Place amount 2.0 of sugar in ((1, 4))
Place amount 2.0 of s